In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [18]:
# from kaggle.competitions import nflrush
import numpy as np
import pandas as pd
import math

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

import datetime
import time

from string import punctuation
import re


from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#Modeling
#https://www.kaggle.com/bestpredict/location-eda-8eb410
import os
TRAIN_ABLE_FALSE=True
if TRAIN_ABLE_FALSE:
    os.environ['CUDA_VISIBLE_DEVICES'] = "1"

import sklearn.metrics as mtr
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda,BatchNormalization
from keras import optimizers
from keras.models import Model
import keras.backend as K
from keras.callbacks import Callback
from  keras.callbacks import EarlyStopping,ModelCheckpoint

TRAIN_OFFLINE = False

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings("ignore")

In [19]:
%%time
train = pd.read_csv('./data/train.csv', dtype={'WindSpeed': 'object'})
train =  pd.DataFrame(train)

Wall time: 6.39 s


In [20]:
#Tools for Preprocessing

def clean_StadiumType(txt):
    if pd.isna(txt):
        return np.nan
    txt = txt.lower()
    txt = ''.join([c for c in txt if c not in punctuation])
    txt = re.sub(' +', ' ', txt)
    txt = txt.strip()
    txt = txt.replace('outside', 'outdoor')
    txt = txt.replace('outdor', 'outdoor')
    txt = txt.replace('outddors', 'outdoor')
    txt = txt.replace('outdoors', 'outdoor')
    txt = txt.replace('oudoor', 'outdoor')
    txt = txt.replace('indoors', 'indoor')
    txt = txt.replace('ourdoor', 'outdoor')
    txt = txt.replace('retractable', 'rtr.')
    return txt

def transform_StadiumType(txt):
    if pd.isna(txt):
        return np.nan
    if 'outdoor' in txt or 'open' in txt:
        return 2
    if 'indoor' in txt or 'closed' in txt:
        return 0
    if 'dome' in txt:
        return 1
    
    return np.nan

Turf = {'Field Turf':1, 'A-Turf Titan':1, 'Grass':0, 'UBU Sports Speed S5-M':1, 
        'Artificial':1, 'DD GrassMaster':1, 'Natural Grass':0, 
        'UBU Speed Series-S5-M':1, 'FieldTurf':1, 'FieldTurf 360':1, 'Natural grass':0, 'grass':0, 
        'Natural':0, 'Artifical':1, 'FieldTurf360':1, 'Naturall Grass':0, 'Field turf':1, 
        'SISGrass':1, 'Twenty-Four/Seven Turf':1, 'natural grass':0} 

def str_to_float(txt):
    try:
        return float(txt)
    except:
        return -1
    
def clean_WindDirection(txt):
    if pd.isna(txt):
        return np.nan
    txt = txt.lower()
    txt = ''.join([c for c in txt if c not in punctuation])
    txt = txt.replace('from', '')
    txt = txt.replace(' ', '')
    txt = txt.replace('north', 'N')
    txt = txt.replace('south', 'S')
    txt = txt.replace('west', 'W')
    txt = txt.replace('east', 'E')
    return txt

def transform_WindDirection(txt):
    if pd.isna(txt):
        return np.nan
    
    if txt=='n':
        return 0
    if txt=='nne' or txt=='nen':
        return 1/8
    if txt=='ne':
        return 2/8
    if txt=='ene' or txt=='nee':
        return 3/8
    if txt=='e':
        return 4/8
    if txt=='ese' or txt=='see':
        return 5/8
    if txt=='se':
        return 6/8
    if txt=='ses' or txt=='sse':
        return 7/8
    if txt=='s':
        return 8/8
    if txt=='ssw' or txt=='sws':
        return 9/8
    if txt=='sw':
        return 10/8
    if txt=='sww' or txt=='wsw':
        return 11/8
    if txt=='w':
        return 12/8
    if txt=='wnw' or txt=='nww':
        return 13/8
    if txt=='nw':
        return 14/8
    if txt=='nwn' or txt=='nnw':
        return 15/8
    return np.nan

def map_weather(txt):
    ans = 1
    if pd.isna(txt):
        return 0
    if 'partly' in txt:
        ans*=0.5
    if 'climate controlled' in txt or 'indoor' in txt:
        return ans*3
    if 'sunny' in txt or 'sun' in txt:
        return ans*2
    if 'clear' in txt:
        return ans
    if 'cloudy' in txt:
        return -ans
    if 'rain' in txt or 'rainy' in txt:
        return -2*ans
    if 'snow' in txt:
        return -3*ans
    return 0

def OffensePersonnelSplit(x):
    dic = {'DB' : 0, 'DL' : 0, 'LB' : 0, 'OL' : 0, 'QB' : 0, 'RB' : 0, 'TE' : 0, 'WR' : 0}
    for xx in x.split(","):
        xxs = xx.split(" ")
        dic[xxs[-1]] = int(xxs[-2])
    return dic

def DefensePersonnelSplit(x):
    dic = {'DB' : 0, 'DL' : 0, 'LB' : 0, 'OL' : 0}
    for xx in x.split(","):
        xxs = xx.split(" ")
        dic[xxs[-1]] = int(xxs[-2])
    return dic

def YardGain(train):
    YG = 0
    if train['YardLine'] == 50:
        YG = 50
    #Home team offense at home field
    elif (train['FieldPosition'] == train['HomeTeamAbbr']) & (train['isOffenseHome'] == True):
        YG = train['YardLine']
    #Home team offense at away field
    elif (train['FieldPosition'] == train['VisitorTeamAbbr']) & (train['isOffenseHome'] == True):
        YG = 100 - train['YardLine']
    #Away team offense at away field
    elif (train['FieldPosition'] == train['VisitorTeamAbbr']) & (train['isOffenseHome'] == False):    
        YG = train['YardLine']
    #Away team offense at home field
    elif (train['FieldPosition'] == train['HomeTeamAbbr']) & (train['isOffenseHome'] == False):
        YG = 100 - train['YardLine']
    return YG

def DistanceFromScrimmageLine(Df):
    Distance = 0
    if Df.PlayDirection == 'left':
        Distance = - (10 + Df['OffenseYardLine'] - (120 - Df['X_Rusher']))
    if Df.PlayDirection == 'right':
        Distance = - (10 + Df['OffenseYardLine'] - (Df['X_Rusher']))
    return Distance

import matplotlib.patches as patches
def PlayVisualization(linenumbers=True,
                          endzones=True,
                          highlight_line=False,
                          highlight_line_number=50,
                          highlighted_name='Line of Scrimmage',
                          fifty_is_los=False,
                          figsize=(12, 6.33)):
    """
    Function that plots the football field for viewing plays.
    Allows for showing or hiding endzones.
    """
    rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1,
                             edgecolor='r', facecolor='darkgreen', zorder=0)

    fig, ax = plt.subplots(1, figsize=figsize)
    ax.add_patch(rect)

    plt.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
              53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
             color='white')
    if fifty_is_los:
        plt.plot([60, 60], [0, 53.3], color='gold')
        plt.text(62, 50, '<- Player Yardline at Snap', color='gold')
    # Endzones
    if endzones:
        ez1 = patches.Rectangle((0, 0), 10, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ez2 = patches.Rectangle((110, 0), 120, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ax.add_patch(ez1)
        ax.add_patch(ez2)
    plt.xlim(0, 120)
    plt.ylim(-5, 58.3)
    plt.axis('off')
    if linenumbers:
        for x in range(20, 110, 10):
            numb = x
            if x > 50:
                numb = 120 - x
            plt.text(x, 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white')
            plt.text(x - 0.95, 53.3 - 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white', rotation=180)
    if endzones:
        hash_range = range(11, 110)
    else:
        hash_range = range(1, 120)

    for x in hash_range:
        ax.plot([x, x], [0.4, 0.7], color='white')
        ax.plot([x, x], [53.0, 52.5], color='white')
        ax.plot([x, x], [22.91, 23.57], color='white')
        ax.plot([x, x], [29.73, 30.39], color='white')

    if highlight_line:
        hl = highlight_line_number + 10
        plt.plot([hl, hl], [0, 53.3], color='yellow')
        plt.text(hl + 2, 50, '<- {}'.format(highlighted_name),
                 color='yellow')
    return fig, ax

def TeamAbbrMapper(Abbr):
    if Abbr == 'ARI':
        return 'ARZ'
    if Abbr == 'BAL':
        return 'BLT'
    if Abbr == 'CLE':
        return 'CLV'
    if Abbr == 'HOU':
        return 'HST'
    else:
        return Abbr

In [21]:
import itertools
def OffenseFormationMapper(Formation):
    Split = Formation.split(",")
    Positions = []
    Nums = []
    
    for s in Split:
        Position = s.split(" ")[-1]
        Positions.append(Position)
        
        Num = s.split(" ")[-2]
        Nums.append(Num)

    PositionLists = []
    for i in range(len(Positions)):
        PositionList = list(itertools.repeat(Positions[i], int(Nums[i])))
        PositionLists = PositionLists + PositionList

    OffenseDic = {'OL' : 0,'QB' : 0,'RB' : 0,'TE' : 0,'WR' : 0}

    OffenseDic['OL'] = PositionLists.count('OL')
    OffenseDic['QB'] = PositionLists.count('QB')
    OffenseDic['RB'] = PositionLists.count('RB')
    OffenseDic['TE'] = PositionLists.count('TE')
    OffenseDic['WR'] = PositionLists.count('WR')
    
    return OffenseDic

In [22]:
def DefenseFormationMapper(Formation):
    Split = Formation.split(",")
    Positions = []
    Nums = []
    
    for s in Split:
        Position = s.split(" ")[-1]
        Positions.append(Position)
        
        Num = s.split(" ")[-2]
        Nums.append(Num)

    PositionLists = []
    for i in range(len(Positions)):
        PositionList = list(itertools.repeat(Positions[i], int(Nums[i])))
        PositionLists = PositionLists + PositionList

    DefenseDic = {'DB' : 0, 'DL' : 0, 'LB' : 0, 'OL' : 0}

    DefenseDic['DB'] = PositionLists.count('DB')
    DefenseDic['DL'] = PositionLists.count('DL')
    DefenseDic['LB'] = PositionLists.count('LB')
    DefenseDic['OL'] = PositionLists.count('OL')
    
    return DefenseDic

In [24]:
def PreprocessingTrainSet(train):    
    #GroupFeatures
#     GroupFeatures = ['Yards_OffenseFormation_mean', 'Yards_DefendersInTheBox_mean', 'Yards_Down_mean', 'Yards_PossessionTeam_mean']

#     temp = train.groupby('OffenseFormation')['Yards'].agg(['mean']).rename({'mean':'Yards_OffenseFormation_mean'},axis=1)
#     train = pd.merge(train,temp,on='OffenseFormation',how='left')

#     temp = train.groupby('DefendersInTheBox')['Yards'].agg(['mean']).rename({'mean':'Yards_DefendersInTheBox_mean'},axis=1)
#     train = pd.merge(train,temp,on='DefendersInTheBox',how='left')

#     temp = train.groupby('Down')['Yards'].agg(['mean']).rename({'mean':'Yards_Down_mean'},axis=1)
#     train = pd.merge(train,temp,on='Down',how='left')

#     temp = train.groupby('PossessionTeam')['Yards'].agg(['mean']).rename({'mean':'Yards_PossessionTeam_mean'},axis=1)
#     train = pd.merge(train,temp,on='PossessionTeam',how='left')

#     GroupFeaturesDf = train[['PlayId'] + GroupFeatures].groupby('PlayId').agg('mean')

    #Ground & Weather Conditions 
    ConditionFeatures = ['StadiumType','Turf','GameWeather','Temperature','Humidity','WindSpeed',
    'WindDirection', 'Stadium', 'Location']
    Drop = ['Stadium','Location']
    Drops = Drop

    train['StadiumType'] = train['StadiumType'].apply(clean_StadiumType)
    train['StadiumType'] = train['StadiumType'].apply(transform_StadiumType)
    train['Turf'] = train['Turf'].map(Turf)

    #WindSpeed
    train['WindSpeed'] = train['WindSpeed'].apply(lambda x: x.lower().replace('mph', '').strip() if not pd.isna(x) else x)
    train['WindSpeed'] = train['WindSpeed'].apply(lambda x: (int(x.split('-')[0])+int(x.split('-')[1]))/2 if not pd.isna(x) and '-' in x else x)
    train['WindSpeed'] = train['WindSpeed'].apply(lambda x: (int(x.split()[0])+int(x.split()[-1]))/2 if not pd.isna(x) and type(x)!=float and 'gusts up to' in x else x)
    train['WindSpeed'] = train['WindSpeed'].apply(str_to_float)

    #WindDirection
    train['WindDirection'] = train['WindDirection'].apply(clean_WindDirection)
    train['WindDirection'] = train['WindDirection'].apply(transform_WindDirection)

    #GameWeather
    train['GameWeather'] = train['GameWeather'].str.lower()
    indoor = "indoor"
    train['GameWeather'] = train['GameWeather'].apply(lambda x: indoor if not pd.isna(x) and indoor in x else x)
    train['GameWeather'] = train['GameWeather'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
    train['GameWeather'] = train['GameWeather'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
    train['GameWeather'] = train['GameWeather'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
    train['GameWeather'] = train['GameWeather'].apply(map_weather)

    #Drop Location
    train = train.drop(columns = Drop)

    #Time Features
    TimeFeatures = ['TimeHandoff','TimeSnap','Season','Quarter','Week']
    Drop = ['TimeHandoff','TimeSnap','GameClock']
    Drops = Drops + Drop

    #Handle Time Data
    train['TimeHandoff'] = train['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    train['TimeSnap'] = train['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    train['PlayerBirthDate'] = train['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))

    #New Features
    train['SnaptoHandoff'] = train.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)
    train['QuarterSecondsLeft'] = train['GameClock'].apply(lambda x: int(x[0:2])*60 + int(x[3:5]))
    train['GameSecondsLeft'] = train['Quarter'].map({1:2700, 2:1800, 3:900, 4:0}) + train['QuarterSecondsLeft']
    train['GameSecondsPassed'] = 3600 - train['GameSecondsLeft']
    train['PlayerAge'] = 2019 - train.PlayerBirthDate.apply(lambda x : x.year)

    #For 5th Quarter
    train.loc[(train['Quarter'] == 5),'GameSecondsLeft'] = -1
    train.loc[(train['Quarter'] == 5),'QuarterSecondsLeft'] = -1
    train.loc[(train['Quarter'] == 5),'GameSecondsPassed'] = 3600

    #Drop useless features
    train = train.drop(columns = Drop)

    #Team Features
    TeamFeatures = ['Team','HomeTeamAbbr','VisitorTeamAbbr','HomeScoreBeforePlay',
    'VisitorScoreBeforePlay','PossessionTeam','FieldPosition','Dis','YardLine','Down','Distance',
    'OffenseFormation','OffensePersonnel','DefendersInTheBox','DefensePersonnel']
    Drop = ['HomeTeamAbbr','VisitorTeamAbbr','HomeScoreBeforePlay','VisitorScoreBeforePlay',
    'PossessionTeam', 'FieldPosition']
    Drops = Drops + Drop

    #Possession
    #To Handle some typos
    map_abbr = {'ARI': 'ARZ', 'BAL': 'BLT', 'CLE': 'CLV', 'HOU': 'HST'}

    for abb in train['PossessionTeam'].unique():
        map_abbr[abb] = abb

    train['PossessionTeam'] = train['PossessionTeam'].map(map_abbr)
    train['HomeTeamAbbr'] = train['HomeTeamAbbr'].map(map_abbr)
    train['VisitorTeamAbbr'] = train['VisitorTeamAbbr'].map(map_abbr)

    #New Features
    train['OffenseTeamAbbr'] = train['PossessionTeam']
    train['DefenseTeamAbbr'] = train.apply(lambda row : row['VisitorTeamAbbr'] if row['OffenseTeamAbbr'] == row['HomeTeamAbbr'] else row['HomeTeamAbbr'], axis = 1)
    train['isOffenseHome'] = train['HomeTeamAbbr'] == train['PossessionTeam']
    train['isOffenseField'] = train['FieldPosition'] == train['PossessionTeam']
    train['OffenseLeadScore'] = train.apply(lambda row : row['HomeScoreBeforePlay'] - row['VisitorScoreBeforePlay'] if row['isOffenseHome'] == 'True' else row['VisitorScoreBeforePlay'] - row['HomeScoreBeforePlay'], axis = 1)
    train['OffenseYardLine'] =  train.apply(YardGain, axis = 1)

    #Strategies & Formations
    FormationFeatures = ['OffensePersonnel','DefendersInTheBox',
                         'DefensePersonnel']
    Drop = ['OffensePersonnel','DefensePersonnel']
    Drops.append(Drop)



    #OffensivePersonnel
    temp = train["OffensePersonnel"].iloc[np.arange(0, len(train), 22)].apply(lambda x : pd.Series(OffenseFormationMapper(x)))
    temp.columns = ["Offense" + c for c in temp.columns]
    temp["PlayId"] = train["PlayId"].iloc[np.arange(0, len(train), 22)]
    train = train.merge(temp, on = 'PlayId')

    #DefensivePersonnel
    temp = train["DefensePersonnel"].iloc[np.arange(0, len(train), 22)].apply(lambda x : pd.Series(DefenseFormationMapper(x)))
    temp.columns = ["Defense" + c for c in temp.columns]
    temp["PlayId"] = train["PlayId"].iloc[np.arange(0, len(train), 22)]
    train = train.merge(temp, on = 'PlayId')

    #Drop
    train = train.drop(columns = Drop)

    #Player Features
    PlayerFeatures = ['Team','DisplayName','JerseyNumber','NflIdRusher','PlayerCollegeName',
    'NflId','X','Y','S','A','Orientation','Dir','PlayerHeight','PlayerWeight','Position',
    'PlayerBirthDate']

    Drop = ['Team','DisplayName','JerseyNumber','NflIdRusher','PlayerCollegeName','NflId',
    'PlayerBirthDate']
    Drops = Drops + Drop

    #Height , Weight , BMI
    train['PlayerHeight'] = train['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))
    train['PlayerBMI'] = 703*(train['PlayerWeight']/(train['PlayerHeight'])**2)

    #Position Cleaning

    train['Position'] = train.Position.replace('DE','DL')
    train['Position'] = train.Position.replace('DT','DL')
    train['Position'] = train.Position.replace('NT','DL')

    train['Position'] = train.Position.replace('ILB','LB')
    train['Position'] = train.Position.replace('MLB','LB')
    train['Position'] = train.Position.replace('OLB','LB')

    train['Position'] = train.Position.replace('CB','DB')

    train['Position'] = train.Position.replace('FS','S')
    train['Position'] = train.Position.replace('SS','S')
    train['Position'] = train.Position.replace('SAF','S')

    train['Position'] = train.Position.replace('HB','RB')

    train['Position'] = train.Position.replace('C','OL')
    train['Position'] = train.Position.replace('G','OL')
    train['Position'] = train.Position.replace('T','OL')
    train['Position'] = train.Position.replace('OT','OL')
    train['Position'] = train.Position.replace('OG','OL')
    train['Position'] = train.Position.replace('OG','OL')

    #Position Encoding
    train['Position'] = train.Position.replace('DL',-4)
    train['Position'] = train.Position.replace('LB',-3)
    train['Position'] = train.Position.replace('DB',-2)
    train['Position'] = train.Position.replace('S',-1)

    train['Position'] = train.Position.replace('RB',6)
    train['Position'] = train.Position.replace('WR',5)
    train['Position'] = train.Position.replace('FB',4)
    train['Position'] = train.Position.replace('QB',3)
    train['Position'] = train.Position.replace('OL',2)
    train['Position'] = train.Position.replace('TE',1)

    #Transform Orientation to Radian
    train['Orientation_rad'] = np.mod(train.Orientation, 360) * math.pi/180.0

    #Fix shifted orientation 
    train.loc[train.Season >= 2018, 'Orientation_rad'] = np.mod(train.loc[train.Season >= 2018, 'Orientation'] - 90, 360) * math.pi/180.0

    #Overwrite Orientation
    train['Orientation'] = train['Orientation_rad']
    train.drop(columns = 'Orientation_rad',inplace = True)

    #PlayerInfo Dataframe
    PlayerInfo = train[['PlayId','DisplayName','NflId','NflIdRusher','X','Y','Position','Team','isOffenseHome','A','S','PlayerAge','PlayerWeight','PlayerHeight','PlayerBMI','Dir','Orientation']]
    PlayerInfo.set_index('PlayId',inplace = True)

    #Create RusherInfo Dataframe
    RusherInfo = train[(train['NflId'] == train['NflIdRusher'])][['PlayId','DisplayName','Position','X','Y','A','S','Dir','Orientation','PlayerAge','PlayerHeight','PlayerWeight','PlayerBMI']].set_index('PlayId')[['DisplayName','Position','X','Y','A','S','Dir','Orientation','PlayerAge','PlayerHeight','PlayerWeight','PlayerBMI']]
    RusherInfo.columns = ['RusherName','RusherPosition','X_Rusher','Y_Rusher','A_Rusher','S_Rusher','Dir_Rusher','Orientation_Rusher','PlayerAge_Rusher','PlayerHeight_Rusher','PlayerWeight_Rusher','PlayerBMI_Rusher']

    #Add Player's performance
    Top10Rusher = ['Christian McCaffrey','Kenyan Drake','Nick Chubb','Dalvin Cook','Ezekiel Elliott',
    'Leonard Fournette','Josh Jacobs','Chris Carson','Marlon Mack','Lamar Jackson']
    RusherInfo['isTop10Rusher'] = RusherInfo['RusherName'].isin(Top10Rusher)

    #Rusher's vertical / horizontal physics
    RadianAngle = (90 - RusherInfo['Dir_Rusher']) * np.pi / 180.0
    RusherInfo['A_Horizontal_Rusher'] = np.abs(RusherInfo['A_Rusher'] * np.cos(RadianAngle))
    RusherInfo['A_Vertical_Rusher'] = np.abs(RusherInfo['A_Rusher'] * np.sin(RadianAngle))

    RusherInfo['S_Horizontal_Rusher'] = np.abs(RusherInfo['S_Rusher'] * np.cos(RadianAngle))
    RusherInfo['S_Vertical_Rusher'] = np.abs(RusherInfo['S_Rusher'] * np.sin(RadianAngle))

    RusherInfo['F_Rusher'] = np.sqrt(RusherInfo['A_Horizontal_Rusher']**2 + RusherInfo['A_Vertical_Rusher']**2) * RusherInfo['PlayerWeight_Rusher']
    RusherInfo['KE_Rusher'] = (1/2) *  RusherInfo['PlayerWeight_Rusher'] * (RusherInfo['S_Horizontal_Rusher']**2 + RusherInfo['S_Vertical_Rusher']**2)

    #Concat PlayerInfo & RusherInfo
    InfoDf = pd.concat([PlayerInfo,RusherInfo],1)

    #Create DistanceFromRusher / isOffense
    InfoDf['DistanceFromRusher'] = np.sqrt((InfoDf['X_Rusher'] - InfoDf['X'])**2 + (InfoDf['Y_Rusher'] - InfoDf['Y'])**2)
    InfoDf['isOffense'] = InfoDf.apply(lambda row : True if ((row['Team'] == 'home') & (row['isOffenseHome'] == True)) | (row['Team'] == 'away') & (row['isOffenseHome'] == False) else False, axis = 1)


    #Nearest distance between defensive team & rusher
    MinDistance = InfoDf[InfoDf['isOffense']==False][['DistanceFromRusher']].groupby('PlayId').agg(min)
    MinDistance.columns = ['NearestDistanceFromRusher']
    DistanceDf = pd.concat([InfoDf,MinDistance],1)

    #Create DefenderInfo Dataframe
    DefensivePosition = [4,-3,-2,-1]
    DefenderInfo = DistanceDf[DistanceDf['isOffense'] == False]
    NearestDefenderInfo = DefenderInfo[DefenderInfo['DistanceFromRusher'] == DefenderInfo['NearestDistanceFromRusher']]

    DistanceDf['A_NearestDefender'] = NearestDefenderInfo['A']
    DistanceDf['S_NearestDefender'] = NearestDefenderInfo['S']
    DistanceDf['Orientation_NearestDefender'] = NearestDefenderInfo['Orientation']
    DistanceDf['Dir_NearestDefender'] = NearestDefenderInfo['Dir']

    RadianAngle = (90 - NearestDefenderInfo['Dir']) * np.pi / 180.0
    DistanceDf['A_Horizontal_NearestDefender'] = np.abs(DistanceDf['A'] * np.cos(RadianAngle))
    DistanceDf['A_Vertical_NearestDefender'] = np.abs(DistanceDf['A'] * np.sin(RadianAngle))

    DistanceDf['S_Horizontal_NearestDefender'] = np.abs(DistanceDf['S'] * np.cos(RadianAngle))
    DistanceDf['S_Vertical_NearestDefender'] = np.abs(DistanceDf['S'] * np.sin(RadianAngle))

    DistanceDf['F_NearestDefender'] = np.sqrt(DistanceDf['A_Horizontal_NearestDefender']**2 + DistanceDf['A_Vertical_NearestDefender']**2) * DistanceDf['PlayerWeight']
    DistanceDf['KE_Rusher'] = (1/2) *  DistanceDf['PlayerWeight'] * (DistanceDf['S_Horizontal_NearestDefender']**2 + DistanceDf['S_Vertical_NearestDefender']**2)

    PlayerFeatures = ['PlayId','Position','X','Y','S','A','Dis','Orientation','Dir','PlayerHeight',
    'PlayerWeight','PlayerAge']

    ToBeDropped = ['GameId','Team','NflId','DisplayName','JerseyNumber','YardLine',
    'PossessionTeam','FieldPosition','HomeScoreBeforePlay','VisitorScoreBeforePlay',
    'RusherName','NflIdRusher','PlayDirection','PlayerBirthDate','PlayerCollegeName',
    'HomeTeamAbbr','VisitorTeamAbbr','isRusher','isOffense']

    NflDf = pd.concat([train[[f for f in list(train.columns) if f not in list(DistanceDf.columns)]],DistanceDf.reset_index().drop(columns = 'PlayId')], 1)

    PlayerDf = NflDf[PlayerFeatures]

    count = PlayerDf.groupby(['PlayId']).cumcount() + 1
    PlayerDf = PlayerDf.set_index(['PlayId', count]).unstack().sort_index(1, level=1)
    PlayerDf = PlayerDf.reset_index()
    PlayerFeatures = list(PlayerDf.columns)

    feats = []
    for i in range(len(PlayerDf.columns)):
        feat = ''.join(list(PlayerDf.columns[i][0] + str(PlayerDf.columns[i][1])))
        feats.append(feat)
    PlayerDf.columns = feats

    OneHotFeatures = ['OffenseTeamAbbr','DefenseTeamAbbr','RusherPosition','OffenseFormation']

    Booleans = ['isOffenseHome','isTop10Rusher','isOffenseField']
    BoolMap = {True : 1, False : 0}

    #Create TeamDf
    TeamFeatures = [f for f in NflDf.columns if f not in ToBeDropped + PlayerFeatures]

    TeamDf = NflDf[TeamFeatures]
    TeamDf['PlayId'] = train['PlayId']

    #PlayerCode Decoding
    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(-4,'DL')
    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(-3,'LB')
    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(-2,'DB')
    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(-1,'S')

    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(6,'RB')
    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(5,'WR')
    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(4,'FB')
    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(3,'QB')
    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(2,'OL')
    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(1,'TE')



    NumDf = TeamDf[[f for f in TeamDf.columns if f not in Booleans + OneHotFeatures]]
    BoolDf = TeamDf[Booleans].stack().map(BoolMap).unstack()

    Dummies = TeamDf[OneHotFeatures]
    #Fill Nulls
    for col in Dummies.columns:
        Dummies[col].fillna(Dummies[col].mode()[0],inplace = True)

    enc = OneHotEncoder(handle_unknown='ignore')
    DummyDf = enc.fit_transform(Dummies)
    DummyDf = pd.DataFrame(DummyDf.toarray())
    DummyDf.columns = enc.get_feature_names()


    TeamDf = pd.concat([NumDf,BoolDf,DummyDf], 1)
    TeamDf = TeamDf.groupby(['PlayId']).agg('mean')

    Df = pd.concat([PlayerDf,TeamDf.reset_index().drop(columns = 'PlayId')],1)

    #Distance From Scrimmage Line
    Df['PlayDirection'] = train.groupby('PlayId')[['PlayDirection']].agg(pd.Series.mode).reset_index().drop(columns = 'PlayId')['PlayDirection']
    Df['DistanceFromScrimmageLine'] = Df.apply(DistanceFromScrimmageLine, axis = 1)

    Df = Df.drop(columns = 'PlayDirection')
    Df = Df.drop(columns = 'PlayId')
    Df = Df[[f for f in Df.columns if f not in list(PlayerDf.columns)]]
    Df.drop(columns = ['X','Y','S','A','Orientation','Dir','PlayerHeight','PlayerWeight','Position','PlayerAge','PlayerBMI'], inplace = True)


    #Fill Nulls : Mode
    for col in Df.columns:
        Df[col].fillna(Df[col].mode()[0],inplace = True)

    return Df, enc

In [25]:
def PreprocessingTestSet(TestDf, enc):
    
    #GroupFeatures
#   GroupFeatures = ['Yards_OffenseFormation_mean', 'Yards_DefendersInTheBox_mean', 'Yards_Down_mean', 'Yards_PossessionTeam_mean']

#   temp = TestDf.groupby('OffenseFormation')['Yards'].agg(['mean']).rename({'mean':'Yards_OffenseFormation_mean'},axis=1)
#   TestDf = pd.merge(TestDf,temp,on='OffenseFormation',how='left')

#   temp = TestDf.groupby('DefendersInTheBox')['Yards'].agg(['mean']).rename({'mean':'Yards_DefendersInTheBox_mean'},axis=1)
#   TestDf = pd.merge(TestDf,temp,on='DefendersInTheBox',how='left')

#   temp = TestDf.groupby('Down')['Yards'].agg(['mean']).rename({'mean':'Yards_Down_mean'},axis=1)
#   TestDf = pd.merge(TestDf,temp,on='Down',how='left')

#   temp = TestDf.groupby('PossessionTeam')['Yards'].agg(['mean']).rename({'mean':'Yards_PossessionTeam_mean'},axis=1)
#   TestDf = pd.merge(TestDf,temp,on='PossessionTeam',how='left')

#   GroupFeaturesDf = TestDf[['PlayId'] + GroupFeatures].groupby('PlayId').agg('mean')
    #Ground & Weather Conditions 
    ConditionFeatures = ['StadiumType','Turf','GameWeather','Temperature','Humidity','WindSpeed',
                         'WindDirection', 'Stadium', 'Location']
    Drop = ['Stadium','Location']
    Drops = Drop

    TestDf['StadiumType'] = TestDf['StadiumType'].apply(clean_StadiumType)
    TestDf['StadiumType'] = TestDf['StadiumType'].apply(transform_StadiumType)
    TestDf['Turf'] = TestDf['Turf'].map(Turf)

    #WindSpeed
    TestDf['WindSpeed'] = TestDf['WindSpeed'].apply(lambda x: x.lower().replace('mph', '').strip() if not pd.isna(x) else x)
    TestDf['WindSpeed'] = TestDf['WindSpeed'].apply(lambda x: (int(x.split('-')[0])+int(x.split('-')[1]))/2 if not pd.isna(x) and '-' in x else x)
    TestDf['WindSpeed'] = TestDf['WindSpeed'].apply(lambda x: (int(x.split()[0])+int(x.split()[-1]))/2 if not pd.isna(x) and type(x)!=float and 'gusts up to' in x else x)
    TestDf['WindSpeed'] = TestDf['WindSpeed'].apply(str_to_float)

    #WindDirection
    TestDf['WindDirection'] = TestDf['WindDirection'].apply(clean_WindDirection)
    TestDf['WindDirection'] = TestDf['WindDirection'].apply(transform_WindDirection)

    #GameWeather
    TestDf['GameWeather'] = TestDf['GameWeather'].str.lower()
    indoor = "indoor"
    TestDf['GameWeather'] = TestDf['GameWeather'].apply(lambda x: indoor if not pd.isna(x) and indoor in x else x)
    TestDf['GameWeather'] = TestDf['GameWeather'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
    TestDf['GameWeather'] = TestDf['GameWeather'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
    TestDf['GameWeather'] = TestDf['GameWeather'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
    TestDf['GameWeather'] = TestDf['GameWeather'].apply(map_weather)

    #Drop Location
    TestDf = TestDf.drop(columns = Drop)

    #Time Features
    TimeFeatures = ['TimeHandoff','TimeSnap','Season','Quarter','Week']
    Drop = ['TimeHandoff','TimeSnap','GameClock']
    Drops = Drops + Drop

    #Handle Time Data
    TestDf['TimeHandoff'] = TestDf['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    TestDf['TimeSnap'] = TestDf['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    TestDf['PlayerBirthDate'] = TestDf['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))

    #New Features
    TestDf['SnaptoHandoff'] = TestDf.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)
    TestDf['QuarterSecondsLeft'] = TestDf['GameClock'].apply(lambda x: int(x[0:2])*60 + int(x[3:5]))
    TestDf['GameSecondsLeft'] = TestDf['Quarter'].map({1:2700, 2:1800, 3:900, 4:0}) + TestDf['QuarterSecondsLeft']
    TestDf['GameSecondsPassed'] = 3600 - TestDf['GameSecondsLeft']
    TestDf['PlayerAge'] = 2019 - TestDf.PlayerBirthDate.apply(lambda x : x.year)

    #For 5th Quarter
    TestDf.loc[(TestDf['Quarter'] == 5),'GameSecondsLeft'] = -1
    TestDf.loc[(TestDf['Quarter'] == 5),'QuarterSecondsLeft'] = -1
    TestDf.loc[(TestDf['Quarter'] == 5),'GameSecondsPassed'] = 3600

    #Drop useless features
    TestDf = TestDf.drop(columns = Drop)

    #Team Features
    TeamFeatures = ['Team','HomeTeamAbbr','VisitorTeamAbbr','HomeScoreBeforePlay',
    'VisitorScoreBeforePlay','PossessionTeam','FieldPosition','Dis','YardLine','Down','Distance',
    'OffenseFormation','OffensePersonnel','DefendersInTheBox','DefensePersonnel']
    Drop = ['HomeTeamAbbr','VisitorTeamAbbr','HomeScoreBeforePlay','VisitorScoreBeforePlay',
    'PossessionTeam', 'FieldPosition']
    Drops = Drops + Drop

    TestDf['PossessionTeam'] = TestDf['PossessionTeam'].apply(TeamAbbrMapper)
    TestDf['HomeTeamAbbr'] = TestDf['HomeTeamAbbr'].apply(TeamAbbrMapper)
    TestDf['VisitorTeamAbbr'] = TestDf['VisitorTeamAbbr'].apply(TeamAbbrMapper)

    #New Features
    TestDf['OffenseTeamAbbr'] = TestDf['PossessionTeam']
    TestDf['DefenseTeamAbbr'] = TestDf.apply(lambda row : row['VisitorTeamAbbr'] if row['OffenseTeamAbbr'] == row['HomeTeamAbbr'] else row['HomeTeamAbbr'], axis = 1)
    TestDf['isOffenseHome'] = TestDf['HomeTeamAbbr'] == TestDf['PossessionTeam']
    TestDf['isOffenseField'] = TestDf['FieldPosition'] == TestDf['PossessionTeam']
    TestDf['OffenseLeadScore'] = TestDf.apply(lambda row : row['HomeScoreBeforePlay'] - row['VisitorScoreBeforePlay'] if row['isOffenseHome'] == 'True' else row['VisitorScoreBeforePlay'] - row['HomeScoreBeforePlay'], axis = 1)
    TestDf['OffenseYardLine'] =  TestDf.apply(YardGain, axis = 1)

    #Strategies & Formations
    FormationFeatures = ['OffensePersonnel','DefendersInTheBox',
                         'DefensePersonnel']
    Drop = ['OffensePersonnel','DefensePersonnel']
    Drops.append(Drop)


    #OffensivePersonnel
    temp = TestDf["OffensePersonnel"].iloc[np.arange(0, len(TestDf), 22)].apply(lambda x : pd.Series(OffenseFormationMapper(x)))
    temp.columns = ["Offense" + c for c in temp.columns]
    temp["PlayId"] = TestDf["PlayId"].iloc[np.arange(0, len(TestDf), 22)]
    TestDf = TestDf.merge(temp, on = "PlayId")

    #DefensivePersonnel
    temp = TestDf["DefensePersonnel"].iloc[np.arange(0, len(TestDf), 22)].apply(lambda x : pd.Series(DefenseFormationMapper(x)))
    temp.columns = ["Defense" + c for c in temp.columns]
    temp["PlayId"] = TestDf["PlayId"].iloc[np.arange(0, len(TestDf), 22)]
    TestDf = TestDf.merge(temp, on = "PlayId")

    #Drop
    TestDf = TestDf.drop(columns = Drop)

    #Player Features
    PlayerFeatures = ['Team','DisplayName','JerseyNumber','NflIdRusher','PlayerCollegeName',
    'NflId','X','Y','S','A','Orientation','Dir','PlayerHeight','PlayerWeight','Position',
    'PlayerBirthDate']

    Drop = ['Team','DisplayName','JerseyNumber','NflIdRusher','PlayerCollegeName','NflId',
    'PlayerBirthDate']
    Drops = Drops + Drop

    #Height , Weight , BMI
    TestDf['PlayerHeight'] = TestDf['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))
    TestDf['PlayerBMI'] = 703*(TestDf['PlayerWeight']/(TestDf['PlayerHeight'])**2)

    #Position Cleaning

    TestDf['Position'] = TestDf.Position.replace('DE','DL')
    TestDf['Position'] = TestDf.Position.replace('DT','DL')
    TestDf['Position'] = TestDf.Position.replace('NT','DL')

    TestDf['Position'] = TestDf.Position.replace('ILB','LB')
    TestDf['Position'] = TestDf.Position.replace('MLB','LB')
    TestDf['Position'] = TestDf.Position.replace('OLB','LB')

    TestDf['Position'] = TestDf.Position.replace('CB','DB')

    TestDf['Position'] = TestDf.Position.replace('FS','S')
    TestDf['Position'] = TestDf.Position.replace('SS','S')
    TestDf['Position'] = TestDf.Position.replace('SAF','S')

    TestDf['Position'] = TestDf.Position.replace('HB','RB')

    TestDf['Position'] = TestDf.Position.replace('C','OL')
    TestDf['Position'] = TestDf.Position.replace('G','OL')
    TestDf['Position'] = TestDf.Position.replace('T','OL')
    TestDf['Position'] = TestDf.Position.replace('OT','OL')
    TestDf['Position'] = TestDf.Position.replace('OG','OL')
    TestDf['Position'] = TestDf.Position.replace('OG','OL')

    #Position Encoding
    TestDf['Position'] = TestDf.Position.replace('DL',-4)
    TestDf['Position'] = TestDf.Position.replace('LB',-3)
    TestDf['Position'] = TestDf.Position.replace('DB',-2)
    TestDf['Position'] = TestDf.Position.replace('S',-1)

    TestDf['Position'] = TestDf.Position.replace('RB',6)
    TestDf['Position'] = TestDf.Position.replace('WR',5)
    TestDf['Position'] = TestDf.Position.replace('FB',4)
    TestDf['Position'] = TestDf.Position.replace('QB',3)
    TestDf['Position'] = TestDf.Position.replace('OL',2)
    TestDf['Position'] = TestDf.Position.replace('TE',1)
    
    #Transform Orientation to Radian
    TestDf['Orientation_rad'] = np.mod(TestDf.Orientation, 360) * math.pi/180.0
    
    #Fix shifted orientation 
    TestDf.loc[TestDf.Season >= 2018, 'Orientation_rad'] = np.mod(TestDf.loc[TestDf.Season >= 2018, 'Orientation'] - 90, 360) * math.pi/180.0
    
    #Overwrite Orientation
    TestDf['Orientation'] = TestDf['Orientation_rad']
    TestDf.drop(columns = 'Orientation_rad',inplace = True)
    

    #PlayerInfo Dataframe
    PlayerInfo = TestDf[['PlayId','DisplayName','NflId','NflIdRusher','X','Y','Position','Team','isOffenseHome','A','S','PlayerAge','PlayerWeight','PlayerHeight','PlayerBMI','Dir','Orientation']]
    PlayerInfo.set_index('PlayId',inplace = True)

    #Create RusherInfo Dataframe
    RusherInfo = TestDf[(TestDf['NflId'] == TestDf['NflIdRusher'])][['PlayId','DisplayName','Position','X','Y','A','S','Dir','Orientation','PlayerAge','PlayerHeight','PlayerWeight','PlayerBMI']].set_index('PlayId')[['DisplayName','Position','X','Y','A','S','Dir','Orientation','PlayerAge','PlayerHeight','PlayerWeight','PlayerBMI']]
    RusherInfo.columns = ['RusherName','RusherPosition','X_Rusher','Y_Rusher','A_Rusher','S_Rusher','Dir_Rusher','Orientation_Rusher','PlayerAge_Rusher','PlayerHeight_Rusher','PlayerWeight_Rusher','PlayerBMI_Rusher']

    #Add Player's performance
    Top10Rusher = ['Christian McCaffrey','Kenyan Drake','Nick Chubb','Dalvin Cook','Ezekiel Elliott',
    'Leonard Fournette','Josh Jacobs','Chris Carson','Marlon Mack','Lamar Jackson']

    RusherInfo['isTop10Rusher'] = RusherInfo['RusherName'].isin(Top10Rusher)
    
    #Rusher's vertical / horizontal physics
    RadianAngle = (90 - RusherInfo['Dir_Rusher']) * np.pi / 180.0
    RusherInfo['A_Horizontal_Rusher'] = np.abs(RusherInfo['A_Rusher'] * np.cos(RadianAngle))
    RusherInfo['A_Vertical_Rusher'] = np.abs(RusherInfo['A_Rusher'] * np.sin(RadianAngle))

    RusherInfo['S_Horizontal_Rusher'] = np.abs(RusherInfo['S_Rusher'] * np.cos(RadianAngle))
    RusherInfo['S_Vertical_Rusher'] = np.abs(RusherInfo['S_Rusher'] * np.sin(RadianAngle))

    RusherInfo['F_Rusher'] = np.sqrt(RusherInfo['A_Horizontal_Rusher']**2 + RusherInfo['A_Vertical_Rusher']**2) * RusherInfo['PlayerWeight_Rusher']
    RusherInfo['KE_Rusher'] = (1/2) *  RusherInfo['PlayerWeight_Rusher'] * (RusherInfo['S_Horizontal_Rusher']**2 + RusherInfo['S_Vertical_Rusher']**2)

    #Concat PlayerInfo & RusherInfo
    InfoDf = pd.concat([PlayerInfo,RusherInfo],1)

    #Create DistanceFromRusher / isOffense
    InfoDf['DistanceFromRusher'] = np.sqrt((InfoDf['X_Rusher'] - InfoDf['X'])**2 + (InfoDf['Y_Rusher'] - InfoDf['Y'])**2)
    InfoDf['isOffense'] = InfoDf.apply(lambda row : True if ((row['Team'] == 'home') & (row['isOffenseHome'] == True)) | (row['Team'] == 'away') & (row['isOffenseHome'] == False) else False, axis = 1)


    #Nearest distance between defensive team & rusher
    MinDistance = InfoDf[InfoDf['isOffense']==False][['DistanceFromRusher']].groupby('PlayId').agg(min)
    MinDistance.columns = ['NearestDistanceFromRusher']
    DistanceDf = pd.concat([InfoDf,MinDistance],1)

    #Create DefenderInfo Dataframe
    DefensivePosition = [4,-3,-2,-1]
    DefenderInfo = DistanceDf[DistanceDf['isOffense'] == False]
    NearestDefenderInfo = DefenderInfo[DefenderInfo['DistanceFromRusher'] == DefenderInfo['NearestDistanceFromRusher']]

    DistanceDf['A_NearestDefender'] = NearestDefenderInfo['A']
    DistanceDf['S_NearestDefender'] = NearestDefenderInfo['S']
    DistanceDf['Orientation_NearestDefender'] = NearestDefenderInfo['Orientation']
    DistanceDf['Dir_NearestDefender'] = NearestDefenderInfo['Dir']
    
    RadianAngle = (90 - NearestDefenderInfo['Dir']) * np.pi / 180.0
    DistanceDf['A_Horizontal_NearestDefender'] = np.abs(DistanceDf['A'] * np.cos(RadianAngle))
    DistanceDf['A_Vertical_NearestDefender'] = np.abs(DistanceDf['A'] * np.sin(RadianAngle))

    DistanceDf['S_Horizontal_NearestDefender'] = np.abs(DistanceDf['S'] * np.cos(RadianAngle))
    DistanceDf['S_Vertical_NearestDefender'] = np.abs(DistanceDf['S'] * np.sin(RadianAngle))

    DistanceDf['F_NearestDefender'] = np.sqrt(DistanceDf['A_Horizontal_NearestDefender']**2 + DistanceDf['A_Vertical_NearestDefender']**2) * DistanceDf['PlayerWeight']
    DistanceDf['KE_Rusher'] = (1/2) *  DistanceDf['PlayerWeight'] * (DistanceDf['S_Horizontal_NearestDefender']**2 + DistanceDf['S_Vertical_NearestDefender']**2)

    PlayerFeatures = ['PlayId','Position','X','Y','S','A','Dis','Orientation','Dir','PlayerHeight',
    'PlayerWeight','PlayerAge']

    ToBeDropped = ['GameId','Team','NflId','DisplayName','JerseyNumber','YardLine',
    'PossessionTeam','FieldPosition','HomeScoreBeforePlay','VisitorScoreBeforePlay',
    'RusherName','NflIdRusher','PlayDirection','PlayerBirthDate','PlayerCollegeName',
    'HomeTeamAbbr','VisitorTeamAbbr','isRusher','isOffense']

    NflDf = pd.concat([TestDf[[f for f in list(TestDf.columns) if f not in list(DistanceDf.columns)]],DistanceDf.reset_index().drop(columns = 'PlayId')], 1)

    PlayerDf = NflDf[PlayerFeatures]

    count = PlayerDf.groupby(['PlayId']).cumcount() + 1
    PlayerDf = PlayerDf.set_index(['PlayId', count]).unstack().sort_index(1, level=1)
    PlayerDf = PlayerDf.reset_index()

    feats = []
    for i in range(len(PlayerDf.columns)):
        feat = ''.join(list(PlayerDf.columns[i][0] + str(PlayerDf.columns[i][1])))
        feats.append(feat)
    PlayerDf.columns = feats

    OneHotFeatures = ['OffenseTeamAbbr','DefenseTeamAbbr','RusherPosition','OffenseFormation']

    Booleans = ['isOffenseHome','isTop10Rusher','isOffenseField']
    BoolMap = {True : 1, False : 0}

    #Create TeamDf
    TeamFeatures = [f for f in NflDf.columns if f not in ToBeDropped + PlayerFeatures]

    TeamDf = NflDf[TeamFeatures]
    TeamDf['PlayId'] = TestDf['PlayId']

    #PlayerCode Decoding
    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(-4,'DL')
    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(-3,'LB')
    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(-2,'DB')
    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(-1,'S')

    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(6,'RB')
    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(5,'WR')
    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(4,'FB')
    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(3,'QB')
    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(2,'OL')
    TeamDf['RusherPosition'] = TeamDf.RusherPosition.replace(1,'TE')



    NumDf = TeamDf[[f for f in TeamDf.columns if f not in Booleans + OneHotFeatures]]
    BoolDf = TeamDf[Booleans].stack().map(BoolMap).unstack()

    Dummies = TeamDf[OneHotFeatures]
    #Fill Nulls
    for col in Dummies.columns:
        Dummies[col].fillna(Dummies[col].mode()[0],inplace = True)

    DummyDf = enc.transform(Dummies)
    DummyDf = pd.DataFrame(DummyDf.toarray())
    DummyDf.columns = enc.get_feature_names()


    TeamDf = pd.concat([NumDf,BoolDf,DummyDf], 1)

    TeamDf = TeamDf.groupby(['PlayId']).agg('mean')
    Df = pd.concat([PlayerDf,TeamDf.reset_index().drop(columns = 'PlayId')],1)

    #Distance From Scrimmage Line
    Df['PlayDirection'] = TestDf.groupby('PlayId')[['PlayDirection']].agg(pd.Series.mode).reset_index().drop(columns = 'PlayId')['PlayDirection']
    Df['DistanceFromScrimmageLine'] = Df.apply(DistanceFromScrimmageLine, axis = 1)

    Df = Df.drop(columns = 'PlayDirection')
    Df = Df.drop(columns = 'PlayId')
    #Df = Df[[f for f in Df.columns if f not in list(PlayerDf.columns)]]
    #Df.drop(columns = ['X','Y','S','A','Orientation','Dir','PlayerHeight','PlayerWeight','Position','PlayerAge','PlayerBMI'], inplace = True)


    #Fill Nulls : Mode
    for col in Df.columns:
        idx = 0
        Mode = pd.DataFrame(Df[col].value_counts(dropna = False)).index[idx]
        while True:
            if np.isnan(Mode) == False:
                Df[col].fillna(Mode, inplace = True)
                break
            else:
                Df[col].fillna(-1, inplace = True)
                break
    
    return Df

In [26]:
#Radam
import keras.optimizers
__all__ = ['RAdam']
class RAdam(keras.optimizers.Optimizer):
    """RAdam optimizer.
    # Arguments
        learning_rate: float >= 0. Learning rate.
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        epsilon: float >= 0. Fuzz factor. If `None`, defaults to `K.epsilon()`.
        decay: float >= 0. Learning rate decay over each update.
        weight_decay: float >= 0. Weight decay for each param.
        amsgrad: boolean. Whether to apply the AMSGrad variant of this
            algorithm from the paper "On the Convergence of Adam and
            Beyond".
        total_steps: int >= 0. Total number of training steps. Enable warmup by setting a positive value.
        warmup_proportion: 0 < warmup_proportion < 1. The proportion of increasing steps.
        min_lr: float >= 0. Minimum learning rate after warmup.
    # References
        - [Adam - A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980v8)
        - [On the Convergence of Adam and Beyond](https://openreview.net/forum?id=ryQu7f-RZ)
        - [On The Variance Of The Adaptive Learning Rate And Beyond](https://arxiv.org/pdf/1908.03265v1.pdf)
    """
    def __init__(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., weight_decay=0., amsgrad=False,
                 total_steps=0, warmup_proportion=0.1, min_lr=0., **kwargs):
        learning_rate = kwargs.pop('lr', learning_rate)
        super(RAdam, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.learning_rate = K.variable(learning_rate, name='learning_rate')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            self.weight_decay = K.variable(weight_decay, name='weight_decay')
            self.total_steps = K.variable(total_steps, name='total_steps')
            self.warmup_proportion = K.variable(warmup_proportion, name='warmup_proportion')
            self.min_lr = K.variable(min_lr, name='min_lr')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.initial_weight_decay = weight_decay
        self.initial_total_steps = total_steps
        self.amsgrad = amsgrad

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr

        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * K.cast(self.iterations, K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1

        if self.initial_total_steps > 0:
            warmup_steps = self.total_steps * self.warmup_proportion
            decay_steps = K.maximum(self.total_steps - warmup_steps, 1)
            decay_rate = (self.min_lr - lr) / decay_steps
            lr = K.switch(
                t <= warmup_steps,
                lr * (t / warmup_steps),
                lr + decay_rate * K.minimum(t - warmup_steps, decay_steps),
            )

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='m_' + str(i)) for (i, p) in enumerate(params)]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='v_' + str(i)) for (i, p) in enumerate(params)]

        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='vhat_' + str(i)) for (i, p) in enumerate(params)]
        else:
            vhats = [K.zeros(1, name='vhat_' + str(i)) for i in range(len(params))]

        self.weights = [self.iterations] + ms + vs + vhats

        beta_1_t = K.pow(self.beta_1, t)
        beta_2_t = K.pow(self.beta_2, t)

        sma_inf = 2.0 / (1.0 - self.beta_2) - 1.0
        sma_t = sma_inf - 2.0 * t * beta_2_t / (1.0 - beta_2_t)

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)

            m_corr_t = m_t / (1.0 - beta_1_t)
            if self.amsgrad:
                vhat_t = K.maximum(vhat, v_t)
                v_corr_t = K.sqrt(vhat_t / (1.0 - beta_2_t))
                self.updates.append(K.update(vhat, vhat_t))
            else:
                v_corr_t = K.sqrt(v_t / (1.0 - beta_2_t))

            r_t = K.sqrt((sma_t - 4.0) / (sma_inf - 4.0) *
                         (sma_t - 2.0) / (sma_inf - 2.0) *
                         sma_inf / sma_t)

            p_t = K.switch(sma_t >= 5, r_t * m_corr_t / (v_corr_t + self.epsilon), m_corr_t)

            if self.initial_weight_decay > 0:
                p_t += self.weight_decay * p

            p_t = p - lr * p_t

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    @property
    def lr(self):
        return self.learning_rate

    @lr.setter
    def lr(self, learning_rate):
        self.learning_rate = learning_rate

    def get_config(self):
        config = {
            'learning_rate': float(K.get_value(self.learning_rate)),
            'beta_1': float(K.get_value(self.beta_1)),
            'beta_2': float(K.get_value(self.beta_2)),
            'decay': float(K.get_value(self.decay)),
            'weight_decay': float(K.get_value(self.weight_decay)),
            'epsilon': self.epsilon,
            'amsgrad': self.amsgrad,
            'total_steps': float(K.get_value(self.total_steps)),
            'warmup_proportion': float(K.get_value(self.warmup_proportion)),
            'min_lr': float(K.get_value(self.min_lr)),
        }
        base_config = super(RAdam, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [27]:
#CRPS 
class CRPSCallBack(Callback):
    
    def __init__(self,validation, predict_batch_size=20, include_on_batch=False):
        super(CRPSCallBack, self).__init__()
        self.validation = validation
        self.predict_batch_size = predict_batch_size
        self.include_on_batch = include_on_batch
        
        print('validation shape',len(self.validation))

    def on_batch_begin(self, batch, logs={}):
        pass

    def on_train_begin(self, logs={}):
        if not ('CRPS_Score_Val' in self.params['metrics']):
            self.params['metrics'].append('CRPS_Score_Val')

    def on_batch_end(self, batch, logs={}):
        if (self.include_on_batch):
            logs['CRPS_Score_Val'] = float('-inf')

    def on_epoch_end(self, epoch, logs={}):
        logs['CRPS_Score_Val'] = float('-inf')
            
        if (self.validation):
            X_valid, y_valid = self.validation[0], self.validation[1]
            y_pred = self.model.predict(X_valid)
            y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
            y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
            Val_Score = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_valid.shape[0])
            Val_Score = np.round(Val_Score, 6)
            logs['CRPS_Score_Val'] = Val_Score

In [28]:
def BuildModel(x_tr,y_tr,x_val,y_val):
    inp = Input(shape = (x_tr.shape[1],))
    x = Dense(1024, input_dim=X.shape[1], activation='relu')(inp)
    x = Dropout(0.7)(x)
    x = BatchNormalization()(x)
    
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.7)(x)
    x = BatchNormalization()(x)
    
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.7)(x)
    x = BatchNormalization()(x)
    
    out = Dense(199, activation='softmax')(x)
    model = Model(inp,out)
    Radam = RAdam()
    Adam = optimizers.Adam()
    model.compile(optimizer=Adam, loss='categorical_crossentropy', metrics=[])
    
    #add lookahead
#     lookahead = Lookahead(k=5, alpha=0.5) # Initialize Lookahead
#     lookahead.inject(model) # add into model

    
    es = EarlyStopping(monitor='CRPS_Score_Val', mode='min',restore_best_weights=True,
                       verbose=1,patience=10)

    #mc = ModelCheckpoint('best_model.h5',monitor='CRPS_score_val',mode='min',
                        #save_best_only=True, verbose=1, save_weights_only=True)
    
    bsz = 1024
    steps = x_tr.shape[0]/bsz
    lr = 1e-05

    model.fit(x_tr, y_tr,callbacks=[CRPSCallBack(validation = (x_val,y_val)),es], epochs=100, batch_size=bsz,verbose=1)
    #model.load_weights("best_model.h5")
    
    y_pred = model.predict(x_val)
    y_valid = y_val
    y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
    val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * x_val.shape[0])
    crps = np.round(val_s, 6)

    return model,crps

def predict(x_te):
    model_num = len(models)
    for k,m in enumerate(models):
        if k==0:
            y_pred = m.predict(x_te,batch_size=1024)
        else:
            y_pred += m.predict(x_te,batch_size=1024)
            
    y_pred = y_pred / model_num
    
    return y_pred

In [29]:
%%time
TrainSet, enc = PreprocessingTrainSet(train)

Wall time: 6min 14s


In [ ]:
%%time
#Training Session
from sklearn.model_selection import train_test_split, KFold
import time

TrainSet, enc = PreprocessingTrainSet(train)

#Split train / val set
X = TrainSet.copy()
X.drop(['Yards'], axis=1, inplace=True)

yards = TrainSet.Yards
y = np.zeros((yards.shape[0], 199))
for idx, target in enumerate(list(yards.astype(int))):
    y[idx][99 + target] = 1

#Scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

#Data Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=12345)

losses = []
models = []
crps_csv = []

s_time = time.time()
yards = TrainSet.Yards

for k in range(2):
    kfold = KFold(5, random_state = 42 + k, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(yards)):
        print("-----------")
        print("-----------")
        tr_x,tr_y = X[tr_inds],y[tr_inds]
        val_x,val_y = X[val_inds],y[val_inds]
        model,crps = BuildModel(tr_x,tr_y,val_x,val_y)
        models.append(model)
        print("the %d fold CRPS : %f"%((k_fold+1),crps))
        crps_csv.append(crps)
 
print("mean CRPS : %f"%np.mean(crps_csv))

In [ ]:
%%time
#Submission
if  TRAIN_OFFLINE==False:
    from kaggle.competitions import nflrush
    env = nflrush.make_env()
    iter_test = env.iter_test()

    for (test_df, sample_prediction_df) in iter_test:
        TestSet = PreprocessingTestSet(test_df,enc)
        TestSet = scaler.transform(TestSet)

        y_pred = predict(TestSet)
        y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1).tolist()[0]

        preds_df = pd.DataFrame(data=[y_pred], columns=sample_prediction_df.columns)
        env.predict(preds_df)

    env.write_submission_file()

In [ ]:
import os
print([filename for filename in os.listdir('/kaggle/working') if '.csv' in filename])